In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np, random, pickle
from itertools import cycle
from sklearn.neighbors import LocalOutlierFactor
%matplotlib inline
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif,SelectKBest,f_classif
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Import Data

In [11]:
#df = pd.read_csv('/content/drive/MyDrive/Masters Project/Gun_Violence_Record_Update.csv')
df = pd.read_csv('/content/drive/MyDrive/Masters_Final/Gun_Violence_Record_Update_Final.csv')

In [12]:
df

,State Code,Region,Age,Gender,Race,Immigrant,Sexual Orientation,Religion,Education,School Performance,...,Leakage How,Leakage Who,Leakage Specific/Nonspecific,Interest in Past Mass Violence,Relationship with Other Shooting(s),Pop Culture Connection,Interest in Firearms,Firearm Proficiency,Criminal Record,label
0,43,0,25.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,...,1.0,1,1.0,0.0,1.0,0.0,1.0,3.0,1.0,1
1,3,3,18.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1
2,38,2,39.0,0.0,0.0,0.0,0.0,1.0,2.0,NaN,...,0.0,0,NaN,0.0,0.0,0.0,1.0,3.0,0.0,1
3,22,0,56.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,0.0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1
4,38,2,31.0,0.0,1.0,0.0,0.0,1.0,2.0,2.0,...,0.0,0,NaN,0.0,0.0,0.0,0.0,3.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,5,3,44.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN,...,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1
177,14,1,19.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,...,NaN,0,NaN,0.0,0.0,2.0,0.0,0.0,1.0,1
178,5,3,57.0,0.0,0.0,0.0,0.0,NaN,2.0,NaN,...,4.0,13,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1
179,22,1,15.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,...,15.0,26,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1


#Preprocess Data

In [13]:
df.replace(r'^\s+$', np.nan, regex=True,  inplace=True)
#df['Kidpping or Hostage Situation'].replace('S', 1,  inplace=True)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(-999999, inplace=True)

#Split train and test set

In [14]:
y = df['label']
X = df.drop(['label'], axis = 1)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size =0.3)

#Customize correlation method

In [16]:
def correlation(dataset, threshold):
    col_corr = set()  
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: 
              colname = corr_matrix.columns[i]                  
              col_corr.add(colname)
    return col_corr 

#Check for Correlation

In [17]:
corr_features = correlation(X_train, 0.85)
corr_features

{'Ability to Perform Daily Tasks',
 'Abusive Behavior',
 'Adult Trauma',
 'Autism Spectrum',
 'Bullied',
 'Bully',
 'Childhood Trauma',
 'Crimil Record',
 'Criminal Record',
 'Emotiolly Abused',
 'Family Member Incarcerated',
 'Firearm Proficiency',
 'Gang Affiliation',
 'Gender',
 'Head Injury / Possible TBI',
 'Health Issues',
 'Highest Level of Crimil Justice Involvement',
 'History of Animal Abuse',
 'History of Domestic Abuse',
 'History of Physical Altercations',
 'History of Sexual Offenses',
 'Immigrant',
 'Increased Agitation',
 'Interest in Firearms',
 'Interest in Past Mass Violence',
 'Isolation',
 'Known Family Mental Health History',
 'Known Hate Group or Chat Room Affiliation',
 'Leakage\xa0',
 'Losing Touch with Reality',
 'Mental Illness',
 'Military Service',
 'Mother Violent Treatment',
 'Motive: Economic Issue',
 'Motive: Employment Issue',
 'Motive: Fame-Seeking',
 'Motive: Homophobia',
 'Motive: Interpersol Conflict\xa0',
 'Motive: Legal Issue',
 'Motive: Misogyny

In [18]:
X_train= X_train.drop(corr_features,axis=1)
X_test= X_test.drop(corr_features,axis=1)
X_train 

,State Code,Region,Age,Race,Religion,Education,School Performance,Birth Order,Number of Siblings,Relationship Status,...,Part II Crimes,Domestic Abuse Specified,Childhood SES,Recent or Ongoing Stressor,Timeline of Signs of Crisis,Substance Use,Known Prejudices,Leakage How,Leakage Who,Leakage Specific/Nonspecific
88,12,3,53.0,0.0,-999999.0,-999999.0,-999999.0,-999999.0,-999999.0,3.0,...,0,0.0,-999999.0,3.0,1.0,1.0,0,0.0,0,-999999.0
12,37,3,26.0,0.0,0.0,-999999.0,2.0,-999999.0,-999999.0,3.0,...,0,4.0,1.0,3.0,1.0,4.0,0,1.0,9,1.0
16,43,0,45.0,0.0,1.0,4.0,2.0,-999999.0,-999999.0,2.0,...,0,5.0,-999999.0,7.0,2.0,0.0,0,0.0,0,-999999.0
156,9,0,19.0,0.0,-999999.0,0.0,0.0,1.0,1.0,0.0,...,10,5.0,2.0,4.0,2.0,3.0,8,5.0,10,1.0
82,6,3,42.0,0.0,1.0,1.0,-999999.0,-999999.0,-999999.0,0.0,...,25,0.0,-999999.0,0.0,3.0,4.0,1,1.0,9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,13,1,36.0,2.0,-999999.0,-999999.0,-999999.0,-999999.0,5.0,0.0,...,14,8.0,-999999.0,2.0,3.0,1.0,0,4.0,5,0.0
103,6,3,24.0,0.0,0.0,2.0,-999999.0,2.0,2.0,0.0,...,6,0.0,2.0,2.0,1.0,0.0,3,2.0,10,0.0
67,4,0,13.0,0.0,1.0,0.0,2.0,1.0,2.0,0.0,...,6,0.0,0.0,1.0,2.0,2.0,0,1.0,7,1.0
24,2,3,39.0,0.0,-999999.0,-999999.0,2.0,-999999.0,1.0,3.0,...,0,0.0,0.0,4.0,-999999.0,0.0,0,0.0,0,-999999.0


#Check for mutual information

In [19]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

#Validate with RandomForest

In [20]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [21]:
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [22]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

26

In [23]:
print(selected_feat)

Index(['State Code', 'Region', 'Age', 'Race', 'Religion', 'Education',
       'School Performance', 'Birth Order', 'Number of Siblings',
       'Relationship Status', 'Children', 'Employment Status',
       'Employment Type ', 'Military Branch', 'Community Involvement',
       'Part I Crimes', 'Part II Crimes', 'Domestic Abuse Specified',
       'Childhood SES', 'Recent or Ongoing Stressor',
       'Timeline of Signs of Crisis', 'Substance Use', 'Known Prejudices ',
       'Leakage How', 'Leakage Who ', 'Leakage Specific/Nonspecific '],
      dtype='object')


In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(50,50))
sns.heatmap(correlation_matrix, cbar=True, fmt='.1f', annot=True, cmap='Blues')
plt.savefig('Correlation Heat map')